In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("HiveConnectionCheck") \
    .config("spark.sql.catalogImplementation","hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
#show all tables in the default databases
spark.sql("SHOW DATABASES").show()

In [36]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



### creating pandas dataframe then spark datafram -> store table into hive -> perform spark sql

In [37]:
#create database
# spark.sql("CREATE DATABASE db_9044")
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  db_1234|
|  db_9044|
|  default|
+---------+



In [ ]:

spark.sql(""" CREATE TABLE db_9044.emp_9044(name STRING,
age INT,
salary DOUBLE)
USING parquet
""")
spark.sql("DESCRIBE FORMATTED db_9044.emp_9044").show(truncate=False)

In [38]:
spark.sql('select * from db_9044.emp_9044').show()

+----+---+------+
|name|age|salary|
+----+---+------+
+----+---+------+



In [39]:
import pandas as pd
data={
    'id':[1,2,3,4],
    'name':["ram","arjun","john","doe"],
    'age':[21,23,34,15]
}
df=pd.DataFrame(data)
spark_df=spark.createDataFrame(df)
spark_df.show()


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|  ram| 21|
|  2|arjun| 23|
|  3| john| 34|
|  4|  doe| 15|
+---+-----+---+



In [40]:
#create table into hive
spark_df.write.mode('overwrite').saveAsTable('db_9044.table_9044')

In [41]:
spark.sql('select * from db_9044.table_9044').show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  2|arjun| 23|
|  3| john| 34|
|  4|  doe| 15|
|  1|  ram| 21|
+---+-----+---+



In [42]:
spark.sql('select * from db_9044.table_9044 where id%2=0').show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  2|arjun| 23|
|  4|  doe| 15|
+---+-----+---+



In [43]:
spark.sql('select *from db_9044.table_9044 where id%2=1').show()

+---+----+---+
| id|name|age|
+---+----+---+
|  3|john| 34|
|  1| ram| 21|
+---+----+---+



In [44]:
#create data frame of Person.json
p_df=spark.read.json('gadgets.json')
p_df.show()

+---------+-------+----+-----------+------------+
|hard disk|  model| ram|screen size|        type|
+---------+-------+----+-----------+------------+
|     null|platina|null|        5.6|mobile phone|
|     null|   bn20|null|        6.2|mobile phone|
|     null|     cz|null|        6.6|mobile phone|
|     null|   xm23|null|        6.5|mobile phone|
|     null|     c3|null|        5.5|mobile phone|
|     null|    pk2|null|        6.5|mobile phone|
|     null|platina|null|        5.0|mobile phone|
|     null|   xm23|null|        5.0|mobile phone|
|     null|     k9|null|        6.5|mobile phone|
|     null|    c12|null|        6.5|mobile phone|
|     null|     k9|null|        5.0|mobile phone|
|     null|platina|null|        6.5|mobile phone|
|     null|   xm23|null|        5.5|mobile phone|
|     null|   bn20|null|        6.5|mobile phone|
|     null|    pk2|null|        6.1|mobile phone|
|     null|     c3|null|        6.0|mobile phone|
|     null|   bn20|null|        6.0|mobile phone|


In [46]:
sp_df = spark.read.option("multiline",True).json("Person.json")
sp_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- company: struct (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |-- phone: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- favouriteFruit: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- index: long (nullable = true)
 |-- isActive: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)

